# SECTION 1: IMPORTS & SETUP

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("="*80)
print("LAST MILE CONNECT - DATA EXPLORATION ")
print("="*80)

LAST MILE CONNECT - DATA EXPLORATION 


# SECTION 2: PATH SETUP

In [2]:
RAW_DATA_PATH = Path("../data/raw")
EXTERNAL_DATA_PATH = Path("../data/external")
PROCESSED_PATH = Path("../data/processed")
VIZ_PATH = Path("../docs/visualizations")

# Create directories if they don't exist
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)
VIZ_PATH.mkdir(parents=True, exist_ok=True)

ENROL_PATH = RAW_DATA_PATH / "enrolment"
BIO_PATH = RAW_DATA_PATH / "biometric"
DEMO_PATH = RAW_DATA_PATH / "demographic"

# SECTION 3: DATA LOADING

In [3]:
# Load enrolment data
enrol_files = list(ENROL_PATH.glob("*.csv"))
enrol_df = pd.concat([pd.read_csv(file) for file in enrol_files], ignore_index=True)
print(f"✅ Enrolment data loaded: {len(enrol_df):,} rows")

# Load biometric data
bio_files = list(BIO_PATH.glob("*.csv"))
bio_df = pd.concat([pd.read_csv(file) for file in bio_files], ignore_index=True)
print(f"✅ Biometric data loaded: {len(bio_df):,} rows")

# Load demographic data
demo_files = list(DEMO_PATH.glob("*.csv"))
demo_df = pd.concat([pd.read_csv(file) for file in demo_files], ignore_index=True)
print(f"✅ Demographic data loaded: {len(demo_df):,} rows")

✅ Enrolment data loaded: 1,006,029 rows
✅ Biometric data loaded: 1,861,108 rows
✅ Demographic data loaded: 2,071,700 rows


# SECTION 4: DATA CLEANING

In [4]:
# Convert dates
for df in [enrol_df, bio_df, demo_df]:
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')

# Check for duplicates BEFORE removing
print(f"\n🔍 Duplicate Check:")
print(f"   Enrolment: {enrol_df.duplicated().sum():,} ({enrol_df.duplicated().sum()/len(enrol_df)*100:.2f}%)")
print(f"   Biometric: {bio_df.duplicated().sum():,} ({bio_df.duplicated().sum()/len(bio_df)*100:.2f}%)")
print(f"   Demographic: {demo_df.duplicated().sum():,} ({demo_df.duplicated().sum()/len(demo_df)*100:.2f}%)")

# Remove duplicates
enrol_df = enrol_df.drop_duplicates(subset=['date', 'state', 'district', 'pincode']).reset_index(drop=True)
bio_df = bio_df.drop_duplicates(subset=['date', 'state', 'district', 'pincode']).reset_index(drop=True)
demo_df = demo_df.drop_duplicates(subset=['date', 'state', 'district', 'pincode']).reset_index(drop=True)

print(f"\n✅ After deduplication:")
print(f"   Enrolment: {len(enrol_df):,} rows")
print(f"   Biometric: {len(bio_df):,} rows")
print(f"   Demographic: {len(demo_df):,} rows")

# Standardize text columns
for df in [enrol_df, bio_df, demo_df]:
    df['state'] = df['state'].str.strip().str.title()
    df['district'] = df['district'].str.strip().str.title()

# Ensure pincode is integer
enrol_df['pincode'] = pd.to_numeric(enrol_df['pincode'], errors='coerce').astype('Int64')
bio_df['pincode'] = pd.to_numeric(bio_df['pincode'], errors='coerce').astype('Int64')
demo_df['pincode'] = pd.to_numeric(demo_df['pincode'], errors='coerce').astype('Int64')

print("\n✅ Data standardization completed")


🔍 Duplicate Check:
   Enrolment: 22,957 (2.28%)
   Biometric: 94,896 (5.10%)
   Demographic: 473,601 (22.86%)

✅ After deduplication:
   Enrolment: 983,072 rows
   Biometric: 1,766,212 rows
   Demographic: 1,598,099 rows

✅ Data standardization completed


# SECTION 5: PINCODE-LEVEL AGGREGATION 

In [5]:
# Aggregate by pincode (for geographic clustering later)
pincode_enrol = enrol_df.groupby(['state', 'district', 'pincode']).agg({
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum'
}).reset_index()

pincode_enrol['total_enrolment'] = (
    pincode_enrol['age_0_5'] + 
    pincode_enrol['age_5_17'] + 
    pincode_enrol['age_18_greater']
)

print(f"\n📍 Pincode-level Summary:")
print(f"   Unique pincodes: {len(pincode_enrol):,}")
print(f"   Total enrolments: {pincode_enrol['total_enrolment'].sum():,}")
print(f"   Avg enrolment per pincode: {pincode_enrol['total_enrolment'].mean():.0f}")

# Save pincode-level data
pincode_enrol.to_csv(PROCESSED_PATH / "pincode_enrolment.csv", index=False)
print("✅ Saved: pincode_enrolment.csv")


📍 Pincode-level Summary:
   Unique pincodes: 28,913
   Total enrolments: 5,331,760
   Avg enrolment per pincode: 184
✅ Saved: pincode_enrolment.csv


# SECTION 6: DISTRICT-LEVEL MONTHLY AGGREGATION

In [7]:
# Aggregate enrolment by district and month
enrol_agg = (
    enrol_df.groupby(['state', 'district', pd.Grouper(key='date', freq='M')])
    .agg({
        'age_0_5': 'sum',
        'age_5_17': 'sum',
        'age_18_greater': 'sum'
    })
    .reset_index()
)

# Aggregate biometric
bio_agg = (
    bio_df.groupby(['state', 'district', pd.Grouper(key='date', freq='M')])
    .agg({
        'bio_age_5_17': 'sum',
        'bio_age_17_': 'sum'
    })
    .reset_index()
)

# Aggregate demographic
demo_agg = (
    demo_df.groupby(['state', 'district', pd.Grouper(key='date', freq='M')])
    .agg({
        'demo_age_5_17': 'sum',
        'demo_age_17_': 'sum'
    })
    .reset_index()
)

# Calculate totals
enrol_agg['total_enrolment'] = (
    enrol_agg['age_0_5'] + 
    enrol_agg['age_5_17'] + 
    enrol_agg['age_18_greater']
)

bio_agg['total_biometric_updates'] = (
    bio_agg['bio_age_5_17'] + 
    bio_agg['bio_age_17_']
)

demo_agg['total_demographic_updates'] = (
    demo_agg['demo_age_5_17'] + 
    demo_agg['demo_age_17_']
)

# Merge all datasets
master_df = enrol_agg.merge(
    bio_agg[['state', 'district', 'date', 'total_biometric_updates']],
    on=['state', 'district', 'date'],
    how='left'
).merge(
    demo_agg[['state', 'district', 'date', 'total_demographic_updates']],
    on=['state', 'district', 'date'],
    how='left'
)

# Fill missing values with 0
master_df[['total_biometric_updates', 'total_demographic_updates']] = (
    master_df[['total_biometric_updates', 'total_demographic_updates']].fillna(0)
)

print(f"\n📊 Master Dataset:")
print(f"   Shape: {master_df.shape}")
print(f"   Unique districts: {master_df.groupby(['state', 'district']).ngroups}")
print(f"   Date range: {master_df['date'].min()} to {master_df['date'].max()}")



📊 Master Dataset:
   Shape: (5004, 9)
   Unique districts: 1045
   Date range: 2025-03-31 00:00:00 to 2025-12-31 00:00:00


# SECTION 7: FEATURE ENGINEERING

In [8]:
# Temporal features
master_df['year'] = master_df['date'].dt.year
master_df['month'] = master_df['date'].dt.month
master_df['quarter'] = master_df['date'].dt.quarter

# Age group proportions
master_df['pct_age_0_5'] = (master_df['age_0_5'] / master_df['total_enrolment'] * 100).fillna(0).round(2)
master_df['pct_age_5_17'] = (master_df['age_5_17'] / master_df['total_enrolment'] * 100).fillna(0).round(2)
master_df['pct_age_18_greater'] = (master_df['age_18_greater'] / master_df['total_enrolment'] * 100).fillna(0).round(2)

# Update rates
master_df['biometric_update_rate'] = (
    master_df['total_biometric_updates'] / master_df['total_enrolment'] * 100
).fillna(0).round(2)

master_df['demographic_update_rate'] = (
    master_df['total_demographic_updates'] / master_df['total_enrolment'] * 100
).fillna(0).round(2)

# Enrolment velocity (month-over-month change)
master_df = master_df.sort_values(['state', 'district', 'date'])
master_df['enrolment_velocity'] = master_df.groupby(['state', 'district'])['total_enrolment'].diff()

print("✅ Feature engineering completed")
print(f"   Total features: {len(master_df.columns)}")

✅ Feature engineering completed
   Total features: 18


# SECTION 8: DESCRIPTIVE STATISTICS

In [9]:
print("\n" + "="*80)
print(" DESCRIPTIVE STATISTICS  ")
print("="*80)

# Overall statistics
print(f"\n📊 OVERALL STATISTICS:")
print(f"   Total enrolments: {master_df['total_enrolment'].sum():,}")
print(f"   Total biometric updates: {master_df['total_biometric_updates'].sum():,}")
print(f"   Total demographic updates: {master_df['total_demographic_updates'].sum():,}")

# District-level statistics
district_totals = master_df.groupby(['state', 'district'])['total_enrolment'].sum()
print(f"\n📍 DISTRICT-LEVEL:")
print(f"   Total districts: {len(district_totals)}")
print(f"   Mean enrolment: {district_totals.mean():,.0f}")
print(f"   Median enrolment: {district_totals.median():,.0f}")
print(f"   Std deviation: {district_totals.std():,.0f}")
print(f"   Min: {district_totals.min():,.0f}")
print(f"   Max: {district_totals.max():,.0f}")

# State-level statistics
state_totals = master_df.groupby('state')['total_enrolment'].sum().sort_values(ascending=False)
print(f"\n🗺️  STATE-LEVEL:")
print(f"   Total states/UTs: {len(state_totals)}")
print(f"\n   Top 10 States by Enrolment:")
for i, (state, count) in enumerate(state_totals.head(10).items(), 1):
    print(f"   {i:2d}. {state:30s} {count:>12,}")

print(f"\n   Bottom 10 States by Enrolment:")
for i, (state, count) in enumerate(state_totals.tail(10).items(), 1):
    print(f"   {i:2d}. {state:30s} {count:>12,}")


 DESCRIPTIVE STATISTICS  

📊 OVERALL STATISTICS:
   Total enrolments: 5,331,760
   Total biometric updates: 40,391,220.0
   Total demographic updates: 27,207,399.0

📍 DISTRICT-LEVEL:
   Total districts: 1045
   Mean enrolment: 5,102
   Median enrolment: 2,817
   Std deviation: 6,428
   Min: 1
   Max: 43,142

🗺️  STATE-LEVEL:
   Total states/UTs: 49

   Top 10 States by Enrolment:
    1. Uttar Pradesh                     1,002,631
    2. Bihar                               593,753
    3. Madhya Pradesh                      487,892
    4. West Bengal                         369,217
    5. Maharashtra                         363,446
    6. Rajasthan                           340,591
    7. Gujarat                             275,042
    8. Assam                               225,359
    9. Karnataka                           219,618
   10. Tamil Nadu                          215,710

   Bottom 10 States by Enrolment:
    1. Lakshadweep                             199
    2. Dadra And Nag

# SECTION 9: DATA QUALITY CHECKS

In [10]:
print("\n" + "="*80)
print(" DATA QUALITY VALIDATION ")
print("="*80)

# Check for missing values
missing = master_df.isnull().sum()
if missing.sum() > 0:
    print("\n⚠️  Missing values found:")
    print(missing[missing > 0])
else:
    print("\n✅ No missing values")

# Check for negative values
numeric_cols = master_df.select_dtypes(include=[np.number]).columns
negative_check = (master_df[numeric_cols] < 0).any()
if negative_check.any():
    print("\n⚠️  Negative values found in:", negative_check[negative_check].index.tolist())
else:
    print("✅ No negative values")

# Check for outliers
Q1 = master_df['total_enrolment'].quantile(0.25)
Q3 = master_df['total_enrolment'].quantile(0.75)
IQR = Q3 - Q1
outliers = master_df[
    (master_df['total_enrolment'] < Q1 - 1.5*IQR) | 
    (master_df['total_enrolment'] > Q3 + 1.5*IQR)
]
print(f"\n📊 Outliers detected: {len(outliers):,} ({len(outliers)/len(master_df)*100:.2f}%)")

# Check for zero enrolments
zero_enrol = master_df[master_df['total_enrolment'] == 0]
print(f"⚠️  Records with zero enrolment: {len(zero_enrol):,} ({len(zero_enrol)/len(master_df)*100:.2f}%)")


 DATA QUALITY VALIDATION 

⚠️  Missing values found:
enrolment_velocity    1045
dtype: int64

⚠️  Negative values found in: ['enrolment_velocity']

📊 Outliers detected: 311 (6.22%)
⚠️  Records with zero enrolment: 0 (0.00%)


# SECTION 10: EXPLORATORY VISUALIZATIONS

In [11]:
# Visualization 1: Monthly Enrolment Trend
monthly_trend = master_df.groupby('date')['total_enrolment'].sum()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(monthly_trend.index, monthly_trend.values, marker='o', linewidth=2, markersize=6)
ax.fill_between(monthly_trend.index, 0, monthly_trend.values, alpha=0.3)
ax.set_title('Monthly Enrolment Trend (Mar-Dec 2025)', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Enrolments', fontsize=12)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(VIZ_PATH / 'monthly_enrolment_trend.png', dpi=300, bbox_inches='tight')
print("✅ Saved: monthly_enrolment_trend.png")
plt.close()

# Visualization 2: Top 15 States
state_enrol = master_df.groupby('state')['total_enrolment'].sum().nlargest(15)

fig, ax = plt.subplots(figsize=(12, 8))
state_enrol.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 15 States by Total Enrolment', fontsize=16, fontweight='bold')
ax.set_xlabel('Total Enrolments', fontsize=12)
ax.set_ylabel('State', fontsize=12)
ax.grid(axis='x', alpha=0.3)
for i, v in enumerate(state_enrol.values):
    ax.text(v, i, f' {v:,.0f}', va='center', fontsize=10)
plt.tight_layout()
plt.savefig(VIZ_PATH / 'top_15_states.png', dpi=300, bbox_inches='tight')
print("✅ Saved: top_15_states.png")
plt.close()

# Visualization 3: Age Group Distribution
age_totals = master_df[['age_0_5', 'age_5_17', 'age_18_greater']].sum()

fig, ax = plt.subplots(figsize=(8, 8))
colors = ['#ff9999', '#66b3ff', '#99ff99']
wedges, texts, autotexts = ax.pie(
    age_totals, 
    labels=['0-5 years', '5-17 years', '18+ years'],
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    textprops={'fontsize': 12}
)
ax.set_title('Enrolment Distribution by Age Group', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(VIZ_PATH / 'age_group_distribution.png', dpi=300, bbox_inches='tight')
print("✅ Saved: age_group_distribution.png")
plt.close()

# Visualization 4: Top 20 Districts
district_enrol = master_df.groupby(['state', 'district'])['total_enrolment'].sum().nlargest(20)

fig, ax = plt.subplots(figsize=(12, 10))
district_labels = [f"{d[1]} ({d[0]})" for d in district_enrol.index]
ax.barh(range(len(district_enrol)), district_enrol.values, color='coral')
ax.set_yticks(range(len(district_enrol)))
ax.set_yticklabels(district_labels, fontsize=10)
ax.set_title('Top 20 Districts by Total Enrolment', fontsize=16, fontweight='bold')
ax.set_xlabel('Total Enrolments', fontsize=12)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(VIZ_PATH / 'top_20_districts.png', dpi=300, bbox_inches='tight')
print("✅ Saved: top_20_districts.png")
plt.close()

# Visualization 5: Enrolment Distribution (Histogram)
fig, ax = plt.subplots(figsize=(12, 6))
master_df['total_enrolment'].hist(bins=50, ax=ax, color='seagreen', edgecolor='black')
ax.set_title('Distribution of Monthly District Enrolments', fontsize=16, fontweight='bold')
ax.set_xlabel('Total Enrolments', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.axvline(master_df['total_enrolment'].mean(), color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {master_df["total_enrolment"].mean():.0f}')
ax.axvline(master_df['total_enrolment'].median(), color='orange', linestyle='--', 
           linewidth=2, label=f'Median: {master_df["total_enrolment"].median():.0f}')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(VIZ_PATH / 'enrolment_distribution.png', dpi=300, bbox_inches='tight')
print("✅ Saved: enrolment_distribution.png")
plt.close()

print("\n✅ All visualizations created successfully")

✅ Saved: monthly_enrolment_trend.png
✅ Saved: top_15_states.png
✅ Saved: age_group_distribution.png
✅ Saved: top_20_districts.png
✅ Saved: enrolment_distribution.png

✅ All visualizations created successfully


# SECTION 11: SAVE PROCESSED DATA

In [12]:
# Save master dataset
master_df.to_csv(PROCESSED_PATH / "master_district_month.csv", index=False)
print("✅ Saved: master_district_month.csv")

# Create district-level summary (total across all months)
district_summary = master_df.groupby(['state', 'district']).agg({
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum',
    'total_enrolment': 'sum',
    'total_biometric_updates': 'sum',
    'total_demographic_updates': 'sum'
}).reset_index()

district_summary.to_csv(PROCESSED_PATH / "district_summary.csv", index=False)
print("✅ Saved: district_summary.csv")

# Create state-level summary
state_summary = master_df.groupby('state').agg({
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum',
    'total_enrolment': 'sum',
    'total_biometric_updates': 'sum',
    'total_demographic_updates': 'sum'
}).reset_index()

state_summary.to_csv(PROCESSED_PATH / "state_summary.csv", index=False)
print("✅ Saved: state_summary.csv")

✅ Saved: master_district_month.csv
✅ Saved: district_summary.csv
✅ Saved: state_summary.csv


# FINAL SUMMARY

In [13]:
print("\n" + "="*80)
print("✨ DATA EXPLORATION COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n📊 Summary:")
print(f"   Total enrolments: {master_df['total_enrolment'].sum():,}")
print(f"   Unique districts: {master_df.groupby(['state', 'district']).ngroups}")
print(f"   Unique states: {master_df['state'].nunique()}")
print(f"   Date range: {master_df['date'].min()} to {master_df['date'].max()}")

print(f"\n📁 Files created:")
print(f"   • data/processed/master_district_month.csv")
print(f"   • data/processed/district_summary.csv")
print(f"   • data/processed/state_summary.csv")
print(f"   • data/processed/pincode_enrolment.csv")
print(f"   • 5 visualization PNG files in docs/visualizations/")


✨ DATA EXPLORATION COMPLETED SUCCESSFULLY!

📊 Summary:
   Total enrolments: 5,331,760
   Unique districts: 1045
   Unique states: 49
   Date range: 2025-03-31 00:00:00 to 2025-12-31 00:00:00

📁 Files created:
   • data/processed/master_district_month.csv
   • data/processed/district_summary.csv
   • data/processed/state_summary.csv
   • data/processed/pincode_enrolment.csv
   • 5 visualization PNG files in docs/visualizations/
